In [ ]:
import argparse
import psycopg2
import h5py
import numpy as np
import matplotlib.pyplot as plt
import os
import csv
import scipy.signal as sg
from allensdk.core.nwb_data_set import NwbDataSet
import pandas as pd
%matplotlib inline

In [ ]:
#create pandas dataframe of specimens that have passed QC from the IVSCC log
cells_df =pd.read_csv('speclist.csv')
#generate a custom report in LIMS 'ivscc data status' and save the report
#create another dataframe from that report
LIMS_df = pd.read_csv('custom_report.csv')

In [ ]:
#build up all the necessary columns in the cells dataframe
columns = ['Noise_sweeps','depol_sweeps','hyper_sweeps','ss_sweep','ls_sweep','ramp_sweep','full_name','nwb_path','specimen_id','Cre+','Transgenic_Line','ephys_result_id','dendrite_type','depol_threshold', 'hyper_threshold', 'ss_threshold', 'ls_threshold','ramp_threshold_sweep']
for i in columns:
    cells_df[i] = None

In [ ]:
#check the dataframe
cells_df.head()


In [ ]:
#create a list to index through
specimen_list = list(cells_df['Labtrax_ID'])

In [ ]:
transfer data from LIMS dataframe to cell dataframe only for cells that have passed QC
for i in range(len(LIMS_df)):
    try:
        j = specimen_list.index(LIMS_df['specimen_name'][i][-15:-3])
        cells_df['Transgenic_Line'][j] = LIMS_df['transgenic_driver_line'][i]
        cells_df['dendrite_type'][j] = LIMS_df['dendrite_type_tag'][i]
        cells_df['Cre+'][j] = LIMS_df['cre_reporter_positive'][i]
        cells_df['ephys_result_id'][j] = LIMS_df['ephys_result_id'][i]
    except ValueError:
        pass

In [ ]:
def LinuxtoWindow(Linuxpath):
    SplitLinux = Linuxpath.split('/')
    #I should be able to link to \\titan, but I can't figure it out, so I mapped //titan/cns to y:    
    WindowsP = os.path.join('y:/','mousecelltypes',SplitLinux[4],SplitLinux[5],'')
    #WindowsP = os.path.join('\\titan','cns','mousecelltypes',SplitLinux[4],SplitLinux[5],'')    
    return (WindowsP)